In [4]:
# Import libraries
import io
from pathlib import Path
import zipfile
import pandas as pd

# Define directories
DATA_DIR = Path("../data")

# Clean traffic volume data
for zip_path in sorted(DATA_DIR.glob("*.zip")):

    year = str(zip_path)[-8:-4]
    dfs = []

    with zipfile.ZipFile(zip_path) as zip_1:
        for zip_name in sorted(zip_1.namelist()):
            with zip_1.open(zip_name) as zip_file:
                with zipfile.ZipFile(io.BytesIO(zip_file.read())) as zip_2:
                    for i, csv_name in enumerate(sorted(zip_2.namelist())):
                        if csv_name.lower().endswith(".csv"):
                            with zip_2.open(csv_name) as csv_file:

                                df = pd.read_csv(csv_file)

                                # Drop irrelavant columns
                                df = df.drop(["NM_REGION", "QT_VOLUME_24HOUR", "CT_ALARM_24HOUR"], axis=1)

                                # Rename column names
                                df = df.rename(columns={"NB_SCATS_SITE": "site_id", 
                                                        "QT_INTERVAL_COUNT": "datetime",
                                                        "NB_DETECTOR": "detector_id", 
                                                        "CT_RECORDS": "working_period_count"})
                                
                                # Remove rows with all non-positive volume values
                                volume_cols = df.filter(regex="^V").columns
                                df = df[(df[volume_cols] > 0).any(axis=1)]

                                # Replace nan and negative volumes with zeros and adjust working period count
                                df = df.fillna(0)
                                df = df.assign()
                                df["negative_volume_count"] = (df[volume_cols] < 0).sum(axis=1)
                                df[volume_cols] = df[volume_cols].where(df[volume_cols] >= 0, 0)
                                df = df.assign(working_period_count_0 = df["working_period_count"] - df["negative_volume_count"])
                                df.drop(columns=["working_period_count", "negative_volume_count"], inplace=True)
                                df.rename(columns={"working_period_count_0": "working_period_count"}, inplace=True)

                                # Convert dtypes
                                df["site_id"] = df["site_id"].astype("int16")
                                df["datetime"] = pd.to_datetime(df["datetime"])
                                df['detector_id'] = df["detector_id"].astype("int8")
                                df['working_period_count'] = df["working_period_count"].astype("int8")
                                df[volume_cols] = df[volume_cols].astype("int16")

                                # Create individual rows for volume values
                                print(f"Length of df: {len(df)}")
                                transformed_dfs = pd.DataFrame()
                                for i in range(len(df)):
                                    row = df.iloc[i:i+1]
                                    row_0 = row[["site_id", "datetime", "detector_id", "working_period_count"]]
                                    transformed_df = row_0.loc[row_0.index.repeat(96)].reset_index(drop=True)
                                    transformed_df["volume"] = row[volume_cols].T.reset_index(drop=True)
                                    transformed_dfs = pd.concat([transformed_dfs, transformed_df])
                                    print(f"Transformed row {i} successfully.")

                                dfs.append(transformed_df.iloc[1:] if i > 0 else df)
                                print(f"Cleaned and added {csv_name} successfully.")
            break
                        
    traffic_df = pd.concat(dfs, ignore_index=True)
    break

Length of df: 39349
Transformed row 0 successfully.
Transformed row 1 successfully.
Transformed row 2 successfully.
Transformed row 3 successfully.
Transformed row 4 successfully.
Transformed row 5 successfully.
Transformed row 6 successfully.
Transformed row 7 successfully.
Transformed row 8 successfully.
Transformed row 9 successfully.
Transformed row 10 successfully.
Transformed row 11 successfully.
Transformed row 12 successfully.
Transformed row 13 successfully.
Transformed row 14 successfully.
Transformed row 15 successfully.
Transformed row 16 successfully.
Transformed row 17 successfully.
Transformed row 18 successfully.
Transformed row 19 successfully.
Transformed row 20 successfully.
Transformed row 21 successfully.
Transformed row 22 successfully.
Transformed row 23 successfully.
Transformed row 24 successfully.
Transformed row 25 successfully.
Transformed row 26 successfully.
Transformed row 27 successfully.
Transformed row 28 successfully.
Transformed row 29 successfully.


KeyboardInterrupt: 

In [ ]:
# Check schema for improper values and datatypes

SCHEMA = {
    "site_id": {
        "dtype": "int",
        "allow_na": False
    },
    "datetime": {
        "dtype": "datetime",
        "allow_na": False
    },
    "detector_id": {
        "dtype": "int",
        "allow_na": False
    },
    "volume": {
        "columns": lambda df: df.filter(regex="^V").columns,
        "dtype": "int",
        "min": 0,
        "allow_na": False,
    },
    "working_period_count": {
        "dtype": "int",
        "min": 0,
        "allow_na": False,
    }
}

def validate_schema(df, schema):
    errors = []

    for name, rules in schema.items():

        # ---- COLUMN GROUP (regex / lambda) ----
        if "columns" in rules:
            cols = rules["columns"](df)

            if len(cols) == 0:
                errors.append(f"{name}: no matching columns found")
                continue

            for col in cols:
                s = df[col]

                if not rules.get("allow_na", True) and s.isna().any():
                    errors.append(f"{col}: contains NaN values")

                if rules.get("dtype") == "int" and not pd.api.types.is_integer_dtype(s):
                    errors.append(f"{col}: not integer dtype")

                if "min" in rules and (s < rules["min"]).any():
                    errors.append(f"{col}: contains values < {rules['min']}")

                if "max" in rules and (s > rules["max"]).any():
                    errors.append(f"{col}: contains values > {rules['max']}")

        # ---- SINGLE COLUMN ----
        else:
            if name not in df.columns:
                errors.append(f"Missing column: {name}")
                continue

            s = df[name]

            if not rules.get("allow_na", True) and s.isna().any():
                errors.append(f"{name}: contains NaN values")

            if rules.get("dtype") == "int" and not pd.api.types.is_integer_dtype(s):
                errors.append(f"{name}: not integer dtype")

            if rules.get("dtype") == "datetime" and not pd.api.types.is_datetime64_any_dtype(s):
                errors.append(f"{name}: not datetime dtype")

            if "min" in rules and (s < rules["min"]).any():
                errors.append(f"{name}: contains values < {rules['min']}")

            if "max" in rules and (s > rules["max"]).any():
                errors.append(f"{name}: contains values > {rules['max']}")

    return errors

errors = validate_schema(traffic_df, SCHEMA)

if errors:
    raise ValueError("Schema validation failed:\n" + "\n".join(errors))

In [59]:
# Transform cleaned traffic volume data

transformed_df = pd.DataFrame()

for i in range(len(traffic_df)):
    row = traffic_df.iloc[i:i+1]
    row_0 = row[["site_id", "datetime", "detector_id", "working_period_count"]]
    df = row_0.loc[row_0.index.repeat(96)].reset_index(drop=True)
    df["volume"] = row[volume_cols].T.reset_index(drop=True)
    transformed_df = pd.concat([transformed_df, df])
    print(f"Transformed row {i} successfully.")

transformed_df["time_period"] = range(len(transformed_df))

Transformed row 0 successfully.
Transformed row 1 successfully.
Transformed row 2 successfully.
Transformed row 3 successfully.
Transformed row 4 successfully.
Transformed row 5 successfully.
Transformed row 6 successfully.
Transformed row 7 successfully.
Transformed row 8 successfully.
Transformed row 9 successfully.
Transformed row 10 successfully.
Transformed row 11 successfully.
Transformed row 12 successfully.
Transformed row 13 successfully.
Transformed row 14 successfully.
Transformed row 15 successfully.
Transformed row 16 successfully.
Transformed row 17 successfully.
Transformed row 18 successfully.
Transformed row 19 successfully.
Transformed row 20 successfully.
Transformed row 21 successfully.
Transformed row 22 successfully.
Transformed row 23 successfully.
Transformed row 24 successfully.
Transformed row 25 successfully.
Transformed row 26 successfully.
Transformed row 27 successfully.
Transformed row 28 successfully.
Transformed row 29 successfully.
Transformed row 30 s

KeyboardInterrupt: 

In [58]:
traffic_df

,site_id,datetime,detector_id,V00,V01,V02,V03,V04,V05,V06,...,V87,V88,V89,V90,V91,V92,V93,V94,V95,working_period_count
0,105,2020-01-01,1,8,6,30,22,15,16,12,...,26,27,20,16,12,14,12,9,10,96
1,105,2020-01-01,2,21,21,54,45,35,38,31,...,41,40,40,28,30,32,31,22,26,96
2,105,2020-01-01,3,13,33,64,51,47,48,30,...,49,45,38,33,25,28,28,21,15,96
3,105,2020-01-01,4,0,3,3,3,4,5,4,...,4,3,4,4,1,2,4,1,1,96
4,105,2020-01-01,5,15,29,68,64,72,65,61,...,52,47,47,30,31,23,30,26,13,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219184,3933,2020-01-31,8,1,0,1,1,1,0,0,...,1,0,1,1,1,0,1,0,2,96
1219185,3933,2020-01-31,9,1,1,1,1,1,0,0,...,0,1,1,1,1,0,1,0,2,96
1219186,3933,2020-01-31,10,1,1,1,1,1,0,0,...,1,1,1,1,1,0,1,0,2,96
1219187,3933,2020-01-31,12,0,0,1,0,0,0,0,...,3,10,3,4,2,0,1,2,3,96
